In [1]:
!pip install torchmetrics
import re
import pandas as pd
import torch
from transformers import BertModel, BertTokenizerFast
from torchmetrics import PearsonCorrCoef
import json
import functools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 11.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("setu4993/LaBSE")
model = BertModel.from_pretrained("setu4993/LaBSE")
model = model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/367 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def split_sentences(text):
  """Splits a given text into sentences using regular expressions."""
  english_sentences = re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s", text)
  return english_sentences

In [ ]:
#text = """Mumbai (/mʊmˈbaɪ/ ⓘ, .mw-parser-output .IPA-label-small{font-size:85%}.mw-parser-output .references .IPA-label-small,.mw-parser-output .infobox .IPA-label-small,.mw-parser-output .navbox .IPA-label-small{font-size:100%}Marathi: , IAST: Muṃbaī; formerly known as Bombay) is the capital city of the Indian state of Maharashtra. Mumbai is the financial capital and the most populous city of India with an estimated population of 12.5 million (1.25 crore). Mumbai is the centre of the Mumbai Metropolitan Region, the sixth-most populous metropolitan area in the world with a population of over 23 million (2.3 crore) living within the Mumbai Metropolitan Region. Mumbai lies on the Konkan coast on the west coast of India and has a deep natural harbour. In 2008, Mumbai was named an alpha world city. Mumbai  has the highest number of billionaires out of any city in India as well as in Asia.", "The seven islands that constitute Mumbai were earlier home to communities of Marathi language-speaking Koli people. For centuries, the seven islands of Bombay were under the control of successive indigenous rulers before being ceded to the Portuguese Empire, and subsequently to the East India Company in 1661, through the dowry of Catherine Braganza when she was married off to Charles II of England. Beginning in 1782, Mumbai was reshaped by the Hornby Vellard project, which undertook reclamation of the area between the seven islands from the Arabian Sea. Along with the construction of major roads and railways, the reclamation project, completed in 1845, transformed Mumbai into a major seaport on the Arabian Sea. Mumbai in the 19th century was characterized by economic and educational development. During the early 20th century it became a strong base for the Indian independence movement. Upon India's independence in 1947 the city was incorporated into Bombay State. In 1960, following the Samyukta Maharashtra Movement, a new state of Maharashtra was created with Mumbai as the capital."""
# english_sentences = re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s", text)
# english_sentences
indian_json = '/content/drive/MyDrive/RS/Input/indian_city_data.json'
foreign_json = '/content/drive/MyDrive/RS/Input/foreign_city_data.json'

with open(indian_json, "r", encoding="utf-8") as f:
    data = json.load(f)
data_indian = pd.DataFrame(data)

with open(foreign_json, "r", encoding="utf-8") as f:
    data = json.load(f)
data_foreign = pd.DataFrame(data)

data_indian["sentences"] = data_indian["paragraphs"].apply(split_sentences)
data_foreign["sentences"] = data_foreign["paragraphs"].apply(split_sentences)

In [ ]:
def get_city_embedding(city_data, max_len=512):
  """Calculates the average sentence embedding for a city's data using BERT, with error handling for empty sentences.

  Args:
      city_data: A list of strings, where each string is a sentence for the city.
      max_len: The maximum sentence length for padding (default: 512).

  Returns:
      A torch.Tensor representing the average sentence embedding for the city, or None if all sentences are empty.
  """
  embeddings = []
  print("No of sentences: ", len(city_data))
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)
  for sentence in city_data:
    # Preprocess text (e.g., tokenization)
    encoded_input = tokenizer(sentence, return_tensors="pt", padding="max_length", truncation=True, max_length=max_len)
    encoded_input = encoded_input.to(device)

    # Check for empty tensors (all zeros)
    if encoded_input["input_ids"].sum().item() == 0:
      # Handle empty sentence (e.g., filter or assign special value)
      continue  # Skip empty sentences (filtering)
      # encoded_input = torch.zeros_like(encoded_input["input_ids"])  # Assign special value (adjust interpretation later)

    # Get sentence embedding using BERT model
    with torch.no_grad():
      #print(encoded_input['input_ids'].shape)
      output = model(**encoded_input)
      sentence_embedding = output.pooler_output
    embeddings.append(sentence_embedding)

  if not embeddings:
    return None  # Return None if all sentences were empty

  city_embedding = torch.mean(torch.stack(embeddings), dim=0)
  return city_embedding.cpu()

In [ ]:
indian_city_embeddings = {}
for city_index, row in data_indian[["city", "sentences"]].iterrows():
  city_name = row["city"]
  sentences = row["sentences"]
  print('City: ', city_name)
  city_embedding = get_city_embedding(sentences)
  if city_embedding is not None:
    #print(f"City: {city_name}, Embedding: {city_embedding}")
    indian_city_embeddings[city_name] = city_embedding
  else:
    print(f"City: {city_name}, All sentences empty!")

foreign_city_embeddings = {}
for city_index, row in data_foreign[["city", "sentences"]].iterrows():
  city_name = row["city"]
  sentences = row["sentences"]
  print('City: ', city_name)
  city_embedding = get_city_embedding(sentences)
  if city_embedding is not None:
    #print(f"City: {city_name}, Embedding: {city_embedding}")
    foreign_city_embeddings[city_name] = city_embedding
  else:
    print(f"City: {city_name}, All sentences empty!")

City:  Mumbai
No of sentences:  488
City:  Pune
No of sentences:  430
City:  Ahmedabad
No of sentences:  328
City:  Surat
No of sentences:  231
City:  Hyderabad
No of sentences:  329
City:  Delhi
No of sentences:  366
City:  Chennai
No of sentences:  372
City:  Bangalore
No of sentences:  383
City:  Kolkata
No of sentences:  373
City:  Lucknow
No of sentences:  435
City:  Thane
No of sentences:  68
City:  Kanpur
No of sentences:  167
City:  Jaipur
No of sentences:  183
City:  Nagpur
No of sentences:  469
City:  Bhopal
No of sentences:  249
City:  Indore
No of sentences:  317
City:  Visakhapatnam
No of sentences:  324
City:  Pimpri-Chinchwad
No of sentences:  212
City:  Ghaziabad
No of sentences:  88
City:  Patna
No of sentences:  377
City:  Vadodara
No of sentences:  220
City:  Ludhiana
No of sentences:  113
City:  Nashik
No of sentences:  123
City:  Agra
No of sentences:  339
City:  Faridabad
No of sentences:  110
City:  Rajkot
No of sentences:  189
City:  Meerut
No of sentences:  279

In [ ]:
import pickle
with open("/content/drive/MyDrive/RS/indian_city_embeddings_cs.pickle", "wb") as f:
  pickle.dump(indian_city_embeddings, f)

with open("/content/drive/MyDrive/RS/foreign_city_embeddings_cs.pickle", "wb") as f:
  pickle.dump(foreign_city_embeddings, f)

print("City embeddings saved")

City embeddings saved


Computing Similarity

In [3]:
import pickle

with open("/content/drive/MyDrive/RS/indian_city_embeddings_cs.pickle", "rb") as f:
  indian_city_embeddings = pickle.load(f)

with open("/content/drive/MyDrive/RS/foreign_city_embeddings_cs.pickle", "rb") as f:
  foreign_city_embeddings = pickle.load(f)

In [4]:
# Activate GPU (if not already done)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)  # Move model to GPU if available

In [5]:
from torchmetrics.functional import pearson_corrcoef
# metric = PearsonCorrCoef()
def compute_similarity(embedding1, embedding2):
  """Calculates the cosine similarity between two embeddings.

  Args:
      embedding1: A torch.Tensor representing the first embedding.
      embedding2: A torch.Tensor representing the second embedding.

  Returns:
      A float representing the cosine similarity between the two embeddings.
  """
  similarity = torch.nn.functional.cosine_similarity(embedding1, embedding2)
  return similarity.item()  # Extract the scalar value

  # Flatten embedding (1x764) -> (764)
  # embedding1 = torch.squeeze(embedding1,dim=0)
  # embedding2 = torch.squeeze(embedding2,dim=0)

  # Normalize
  # norm1 = torch.linalg.norm(embedding1)
  # embedding1 = embedding1 / norm1

  # norm2 = torch.linalg.norm(embedding2)
  # embedding2 = embedding2 / norm2

  # similarity = pearson_corrcoef(embedding1, embedding2)
  # Access the first element (scalar value) of the tensor
  # return torch.linalg.norm(embedding1 - embedding2)

In [6]:
def find_top_similar_cities(indian_embedding, european_embeddings, european_city_names, top_k=5):
  """Finds the top K most similar European cities to a given Indian city based on embedding similarity.

  Args:
      indian_embedding: A torch.Tensor representing the embedding of the Indian city.
      european_embeddings: A torch.Tensor containing the embeddings of all European cities.
      european_city_names: A list of strings containing the names of all European cities.
      top_k: An integer specifying the number of top similar cities to return (default: 5).

  Returns:
      A list of tuples containing (city_name, similarity) for the top K most similar European cities.
  """
  similarities = []
  indian_embedding = indian_embedding.to(device)
  for i, european_embedding in enumerate(european_embeddings):
    european_embedding = european_embedding.to(device)

    similarity = compute_similarity(indian_embedding, european_embedding)
    similarities.append((european_city_names[i], similarity))
  similarities.sort(key=lambda x: x[1], reverse=True)  # Sort by similarity (descending)
  return similarities[:top_k]

In [7]:
european_city_embeddings = torch.stack([value for value in foreign_city_embeddings.values()])  # Convert to tensor on GPU if available
european_city_names = list(foreign_city_embeddings.keys())

In [ ]:
for indian_city, indian_embedding in indian_city_embeddings.items():
  top_similar_cities = find_top_similar_cities(indian_embedding, european_city_embeddings, european_city_names, 10)
  print(f"City: {indian_city}")
  for city_name, similarity in top_similar_cities:
    print(f"\t- {city_name}: {similarity:.4f}")  # Format similarity to 4 decimal places

City: Mumbai
	- Birmingham: 0.8503
	- Budapest: 0.8502
	- Sofia: 0.8492
	- Bucharest: 0.8424
	- Manchester: 0.8411
	- Bilbao: 0.8372
	- Barcelona: 0.8327
	- Madrid: 0.8305
	- London: 0.8257
	- Constanța: 0.8234
City: Pune
	- Budapest: 0.8592
	- Poznań: 0.8584
	- Bucharest: 0.8482
	- Sofia: 0.8409
	- Porto: 0.8375
	- Prague: 0.8355
	- Cluj-Napoca: 0.8327
	- Plovdiv: 0.8310
	- Milton_Keynes: 0.8287
	- Barcelona: 0.8262
City: Ahmedabad
	- Amsterdam: 0.8702
	- Budapest: 0.8525
	- Bucharest: 0.8409
	- Rotterdam: 0.8403
	- Zagreb: 0.8394
	- Madrid: 0.8390
	- Sofia: 0.8377
	- Manchester: 0.8354
	- Hamburg: 0.8340
	- Birmingham: 0.8322
City: Surat
	- Sofia: 0.8314
	- Bucharest: 0.8116
	- Setúbal: 0.8048
	- Porto: 0.8037
	- Constanța: 0.8036
	- Budapest: 0.7991
	- Rotterdam: 0.7982
	- Utrecht: 0.7979
	- Lucerne: 0.7938
	- Amsterdam: 0.7933
City: Hyderabad
	- Budapest: 0.8519
	- Sofia: 0.8492
	- Madrid: 0.8465
	- Zagreb: 0.8456
	- Granada: 0.8417
	- Heidelberg: 0.8402
	- Dublin: 0.8372
	- Edinbu

In [8]:
indian_city = 'Agra'
embedding = indian_city_embeddings[indian_city]
top_similar_cities = find_top_similar_cities(embedding, european_city_embeddings, european_city_names, 50)
print(f"City: {indian_city}")
for city_name, similarity in top_similar_cities:
    print(f"\t- {city_name}: {similarity:.4f}")  # Format similarity to 4 decimal places

City: Agra
	- Granada: 0.8612
	- Zagreb: 0.8398
	- Prague: 0.8378
	- Berat: 0.8340
	- Évora: 0.8311
	- Córdoba: 0.8235
	- Budapest: 0.8230
	- Amsterdam: 0.8214
	- Sofia: 0.8180
	- Zaragoza: 0.8171
	- Székesfehérvár: 0.8168
	- Cluj-Napoca: 0.8136
	- Edinburgh: 0.8105
	- Madrid: 0.8092
	- Plovdiv: 0.8082
	- Groningen: 0.8067
	- Kraków: 0.8061
	- Graz: 0.8051
	- Bucharest: 0.8047
	- Heidelberg: 0.8047
	- Utrecht: 0.8043
	- Nitra: 0.8042
	- Ghent: 0.8016
	- Durham: 0.8011
	- Valletta: 0.8011
	- Bilbao: 0.8010
	- Sintra: 0.8009
	- Warsaw: 0.7970
	- Szeged: 0.7965
	- Seville: 0.7957
	- Avignon: 0.7956
	- Porto: 0.7954
	- Lisbon: 0.7947
	- Ljubljana: 0.7943
	- Constanța: 0.7929
	- Essen: 0.7928
	- Hamburg: 0.7918
	- Barcelona: 0.7918
	- Erlangen: 0.7916
	- Dublin: 0.7898
	- Paris: 0.7895
	- Ostrava: 0.7894
	- Birmingham: 0.7894
	- Manchester: 0.7886
	- Rijeka: 0.7873
	- Igualada: 0.7865
	- Bordeaux: 0.7857
	- Interlaken: 0.7852
	- Turin: 0.7837
	- Nice: 0.7834


In [ ]:
print(indian_city_embeddings['Mumbai'])